In [ ]:
import os
import pickle
!pip install numpy==1.16.4
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import torch
import torch.nn as nn

from tensorflow import nn
from tensorflow import function, GradientTape, sqrt, abs, reduce_mean, ones_like, zeros_like, convert_to_tensor,float32
from tensorflow import data as tfdata
from tensorflow import config as tfconfig
from tensorflow.keras import Model, Sequential, Input
from tensorflow.keras.layers import GRU, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from datetime import datetime
!pip install transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from IPython.display import Image
from tqdm import tqdm, trange

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.3 MB 1.4 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 0.20.2 requires numpy>=1.18, but you have numpy 1.16.4 which is incompatible.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.16.4 which is incompatible.
tensorflow 2.9.2 requires numpy>=1.20, but you have numpy 1.16.4 which is incompatible.
tables 3.7.0 requires numpy>=1.19.0, but you have numpy 1.16.4 which is incompatible.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.16.4 which is incompatible.
scikit-image 0.18.3 requires numpy>=1.16.5, but you have numpy 1.16.4 wh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.5 MB/s 
     |████████████████████████████████| 15.7 MB 57.5 MB/s 
     |████████████████████████████████| 7.6 MB 47.4 MB/s 
     |████████████████████████████████| 182 kB 55.8 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4


In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "4"

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/gdrive/MyDrive/BOAZ/data.csv', encoding='cp949')
data.head()

,Unnamed: 0,date,close,diff,open,high,low,volume,updown,등락률,날짜,언론사,제목,링크
0,0,2021-11-01,87900.0,8100.0,81600.0,88600.0,77500.0,9173459.0,-0.071672,0.0,2021-11-01,매경이코노미,‘오딘 개발사’ 품었다…카카오게임즈 주가 10% 급등,https://www.mk.co.kr/economy/view/2021/1034761
1,1,2021-11-01,87900.0,8100.0,81600.0,88600.0,77500.0,9173459.0,-0.071672,0.0,2021-11-01,국민일보,"카카오게임즈, ‘오딘’ 개발사 품는다",http://news.kmib.co.kr/article/view.asp?arcid=...
2,2,2021-11-01,87900.0,8100.0,81600.0,88600.0,77500.0,9173459.0,-0.071672,0.0,2021-11-01,연합뉴스,"카카오게임즈, '오딘' 개발사 라이온하트 최대주주 된다(종합)",http://yna.kr/AKR20211101072551002?did=1195m
3,3,2021-11-01,87900.0,8100.0,81600.0,88600.0,77500.0,9173459.0,-0.071672,0.0,2021-11-01,조선비즈,"카카오게임즈, 2090억원 규모의 유상증자 결정",https://biz.chosun.com/stock/stock_general/202...
4,4,2021-11-01,87900.0,8100.0,81600.0,88600.0,77500.0,9173459.0,-0.071672,0.0,2021-11-01,한국경제TV,"""게임주 극성수기 온다""…기관은 카카오게임즈 `찜` [박해린의 뉴스&마켓]",https://www.wowtv.co.kr/NewsCenter/News/Read?a...


In [ ]:
# 입력값을 리스트로 변환

texts = list(data['제목'])
texts[:4]

['‘오딘 개발사’ 품었다…카카오게임즈 주가 10% 급등',
 '카카오게임즈, ‘오딘’ 개발사 품는다',
 "카카오게임즈, '오딘' 개발사 라이온하트 최대주주 된다(종합)",
 '카카오게임즈, 2090억원 규모의 유상증자 결정']

In [ ]:
# 사전학습 모델 로드

tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")
model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC").to(device)

Downloading:   0%|          | 0.00/372 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/143k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/294k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/881 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406M [00:00<?, ?B/s]

In [ ]:
tokenizer.tokenize(texts[0])

['‘',
 '오',
 '##딘',
 '개발',
 '##사',
 '’',
 '품',
 '##었다',
 '…',
 '카카오',
 '##게임',
 '##즈',
 '주가',
 '10',
 '%',
 '급',
 '##등']

In [ ]:
# 사전학습 모델에 데이터 입력
# output 뽑는데 두 시간 반 걸리는 거 다섯번째 시도중이라... 파일로 저장햇슴다

"""
outputs_list = []
for text in tqdm(texts) :
    inputs = tokenizer(text, return_tensors='pt').to(device)
    output = model(**inputs)
    output = output.logits.tolist()[0]
    outputs_list.append(output)
"""

"\noutputs_list = []\nfor text in tqdm(texts) :\n    inputs = tokenizer(text, return_tensors='pt').to(device)\n    output = model(**inputs)\n    output = output.logits.tolist()[0]\n    outputs_list.append(output)\n"

In [ ]:
# output 뽑는데 두 시간 반 걸리는 거 다섯번째 시도중이라... 파일로 저장햇슴다
"""
import csv 

fields = [1, 0, -1]  
    
rows = outputs_list
  
with open('outputs.csv', 'w',newline='') as f: 
      
    write = csv.writer(f) 
      
    write.writerow(fields) 
    write.writerows(rows)
"""

"\nimport csv \n\nfields = [1, 0, -1]  \n    \nrows = outputs_list\n  \nwith open('outputs.csv', 'w',newline='') as f: \n      \n    write = csv.writer(f) \n      \n    write.writerow(fields) \n    write.writerows(rows)\n"

In [ ]:
outputs_df = pd.read_csv('/content/gdrive/MyDrive/BOAZ/outputs.csv')
outputs_df.columns = ['부정', '중립', '긍정']
outputs_df.head()

,부정,중립,긍정
0,-3.296973,-0.422619,3.997941
1,-1.945200,2.858707,-0.666647
2,-3.164887,-2.894524,6.382129
3,-3.261262,6.670206,-2.643503
4,0.265307,-2.193004,2.158594


In [ ]:
# 출력값을 텐서로 변환

outputs = torch.from_numpy(outputs_df.values)
outputs

tensor([[-3.2970, -0.4226,  3.9979],
        [-1.9452,  2.8587, -0.6666],
        [-3.1649, -2.8945,  6.3821],
        ...,
        [-0.2113,  4.5869, -4.1931],
        [-2.9173,  6.3047, -2.6630],
        [-2.8819,  6.5233, -3.0104]], dtype=torch.float64)

In [ ]:
# 출력값을 확률 형태로 변환

predictions = torch.nn.functional.softmax(outputs, dim=-1)
predictions

tensor([[6.7046e-04, 1.1877e-02, 9.8745e-01],
        [7.9003e-03, 9.6373e-01, 2.8373e-02],
        [7.1402e-05, 9.3568e-05, 9.9984e-01],
        ...,
        [8.1761e-03, 9.9167e-01, 1.5251e-04],
        [9.8819e-05, 9.9977e-01, 1.2743e-04],
        [8.2279e-05, 9.9985e-01, 7.2360e-05]], dtype=torch.float64)

In [ ]:
# 출력값 확인

df_sc = pd.DataFrame(predictions.detach().numpy())
df_sc.columns = ['부정','중립','긍정']
df_sc

,부정,중립,긍정
0,0.000670,0.011877,0.987453
1,0.007900,0.963726,0.028373
2,0.000071,0.000094,0.999835
3,0.000049,0.999861,0.000090
4,0.129421,0.011076,0.859504
...,...,...,...
54915,0.000238,0.999523,0.000238
54916,0.004071,0.995791,0.000138
54917,0.008176,0.991671,0.000153
54918,0.000099,0.999774,0.000127


In [ ]:
data = pd.concat([data, df_sc], axis=1)
data

,Unnamed: 0,date,close,diff,open,high,low,volume,updown,등락률,날짜,언론사,제목,링크,부정,중립,긍정
0,0,2021-11-01,87900.0,8100.0,81600.0,88600.0,77500.0,9173459.0,-0.071672,0.0,2021-11-01,매경이코노미,‘오딘 개발사’ 품었다…카카오게임즈 주가 10% 급등,https://www.mk.co.kr/economy/view/2021/1034761,0.000670,0.011877,0.987453
1,1,2021-11-01,87900.0,8100.0,81600.0,88600.0,77500.0,9173459.0,-0.071672,0.0,2021-11-01,국민일보,"카카오게임즈, ‘오딘’ 개발사 품는다",http://news.kmib.co.kr/article/view.asp?arcid=...,0.007900,0.963726,0.028373
2,2,2021-11-01,87900.0,8100.0,81600.0,88600.0,77500.0,9173459.0,-0.071672,0.0,2021-11-01,연합뉴스,"카카오게임즈, '오딘' 개발사 라이온하트 최대주주 된다(종합)",http://yna.kr/AKR20211101072551002?did=1195m,0.000071,0.000094,0.999835
3,3,2021-11-01,87900.0,8100.0,81600.0,88600.0,77500.0,9173459.0,-0.071672,0.0,2021-11-01,조선비즈,"카카오게임즈, 2090억원 규모의 유상증자 결정",https://biz.chosun.com/stock/stock_general/202...,0.000049,0.999861,0.000090
4,4,2021-11-01,87900.0,8100.0,81600.0,88600.0,77500.0,9173459.0,-0.071672,0.0,2021-11-01,한국경제TV,"""게임주 극성수기 온다""…기관은 카카오게임즈 `찜` [박해린의 뉴스&마켓]",https://www.wowtv.co.kr/NewsCenter/News/Read?a...,0.129421,0.011076,0.859504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54915,54915,2022-11-16,44800.0,150.0,44900.0,45450.0,44500.0,466659.0,0.002232,1.0,2022-11-20,테크M,나흘간의 대장정 마친 '지스타 2022'...18만명이 안전하게 '게임문화' 즐겼다?,https://www.techm.kr/news/articleView.html?idx...,0.000238,0.999523,0.000238
54916,54916,2022-11-16,44800.0,150.0,44900.0,45450.0,44500.0,466659.0,0.002232,1.0,2022-11-20,디스이즈게임,지스타 2022 폐막! 4일간 총 관람객 18만 4천여명으로 잠정 집계,https://www.thisisgame.com/webzine/game/nboard...,0.004071,0.995791,0.000138
54917,54917,2022-11-16,44800.0,150.0,44900.0,45450.0,44500.0,466659.0,0.002232,1.0,2022-11-20,그린포스트코리아,'지스타 2022' 성황리 폐막…나흘간 18만명 방문,http://www.greenpostkorea.co.kr/news/articleVi...,0.008176,0.991671,0.000153
54918,54918,2022-11-16,44800.0,150.0,44900.0,45450.0,44500.0,466659.0,0.002232,1.0,2022-11-20,포모스,"[지스타]""게임으로 뭉쳤다""...18살 '지스타' 성숙한 게임 축제로 거듭나",https://www.fomos.kr/redirect/news_view?news_c...,0.000099,0.999774,0.000127


In [ ]:
# 감성수치/반응도 산출

data_sc = data.copy()

data_sc['감성수치'] = np.where(data_sc['부정'] >= data_sc['긍정'], -data_sc['부정'], data_sc['긍정'])
data_sc['반응도'] = np.where(data_sc['부정'] >= data_sc['긍정'], data_sc['부정'], data_sc['긍정'])
data_sc

,Unnamed: 0,date,close,diff,open,high,low,volume,updown,등락률,날짜,언론사,제목,링크,부정,중립,긍정,감성수치,반응도
0,0,2021-11-01,87900.0,8100.0,81600.0,88600.0,77500.0,9173459.0,-0.071672,0.0,2021-11-01,매경이코노미,‘오딘 개발사’ 품었다…카카오게임즈 주가 10% 급등,https://www.mk.co.kr/economy/view/2021/1034761,0.000670,0.011877,0.987453,0.987453,0.987453
1,1,2021-11-01,87900.0,8100.0,81600.0,88600.0,77500.0,9173459.0,-0.071672,0.0,2021-11-01,국민일보,"카카오게임즈, ‘오딘’ 개발사 품는다",http://news.kmib.co.kr/article/view.asp?arcid=...,0.007900,0.963726,0.028373,0.028373,0.028373
2,2,2021-11-01,87900.0,8100.0,81600.0,88600.0,77500.0,9173459.0,-0.071672,0.0,2021-11-01,연합뉴스,"카카오게임즈, '오딘' 개발사 라이온하트 최대주주 된다(종합)",http://yna.kr/AKR20211101072551002?did=1195m,0.000071,0.000094,0.999835,0.999835,0.999835
3,3,2021-11-01,87900.0,8100.0,81600.0,88600.0,77500.0,9173459.0,-0.071672,0.0,2021-11-01,조선비즈,"카카오게임즈, 2090억원 규모의 유상증자 결정",https://biz.chosun.com/stock/stock_general/202...,0.000049,0.999861,0.000090,0.000090,0.000090
4,4,2021-11-01,87900.0,8100.0,81600.0,88600.0,77500.0,9173459.0,-0.071672,0.0,2021-11-01,한국경제TV,"""게임주 극성수기 온다""…기관은 카카오게임즈 `찜` [박해린의 뉴스&마켓]",https://www.wowtv.co.kr/NewsCenter/News/Read?a...,0.129421,0.011076,0.859504,0.859504,0.859504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54915,54915,2022-11-16,44800.0,150.0,44900.0,45450.0,44500.0,466659.0,0.002232,1.0,2022-11-20,테크M,나흘간의 대장정 마친 '지스타 2022'...18만명이 안전하게 '게임문화' 즐겼다?,https://www.techm.kr/news/articleView.html?idx...,0.000238,0.999523,0.000238,-0.000238,0.000238
54916,54916,2022-11-16,44800.0,150.0,44900.0,45450.0,44500.0,466659.0,0.002232,1.0,2022-11-20,디스이즈게임,지스타 2022 폐막! 4일간 총 관람객 18만 4천여명으로 잠정 집계,https://www.thisisgame.com/webzine/game/nboard...,0.004071,0.995791,0.000138,-0.004071,0.004071
54917,54917,2022-11-16,44800.0,150.0,44900.0,45450.0,44500.0,466659.0,0.002232,1.0,2022-11-20,그린포스트코리아,'지스타 2022' 성황리 폐막…나흘간 18만명 방문,http://www.greenpostkorea.co.kr/news/articleVi...,0.008176,0.991671,0.000153,-0.008176,0.008176
54918,54918,2022-11-16,44800.0,150.0,44900.0,45450.0,44500.0,466659.0,0.002232,1.0,2022-11-20,포모스,"[지스타]""게임으로 뭉쳤다""...18살 '지스타' 성숙한 게임 축제로 거듭나",https://www.fomos.kr/redirect/news_view?news_c...,0.000099,0.999774,0.000127,0.000127,0.000127


In [ ]:
# 일자 별 평균

df_sc = data_sc.groupby(['date'])[['감성수치','반응도']].mean()
df_sc = pd.DataFrame(df_sc)
df_sc.reset_index(inplace=True)
df_sc

,date,감성수치,반응도
0,2021-11-01,0.455454,0.499440
1,2021-11-02,0.486631,0.524840
2,2021-11-03,0.721063,0.817289
3,2021-11-04,0.487221,0.663747
4,2021-11-05,0.288234,0.569376
...,...,...,...
253,2022-11-10,-0.292049,0.533644
254,2022-11-11,0.312316,0.486377
255,2022-11-14,0.010467,0.411570
256,2022-11-15,0.371565,0.483728
